# Notebook 2: Genealogies and sequence variation


### Topic outline:
1. A visual introduction to genealogical variation. 
2. Connecting genealogies (unobserved) to genetic variation (observations)


### Learning objectives: 
By the end of this notebook series you should:
1. Be familiar with the concept of genealogies of genes.
2. Understand how mutations occurring on genealogies generates genetic diversity. 


### Additional recommended reading:

- [Rosenberg and Nordborg (2002) Genealogical trees, coalescent theory and the analysis of genetic polymorphisms. Nature Review Genetics](https://eaton-lab.org/slides/genomics/readings/Rosenberg-and-Nordborg-2002.pdf)

In [ ]:
import ipcoal
import toytree
import itertools
import numpy as np

### Genealogies
A fundamental and often confusing aspect of the [coalescent model](https://en.wikipedia.org/wiki/Coalescent_theory) is understanding *what we mean* when referring to a **genealogy**. In its simplest form, a genealogy is any history of ancestor-descendant relationships. But in the context of the coalescent, we are specifically interested in the genealogies of haploid gene copies, not of diploid individuals. Individuals have genealogies (e.g., you and your family) and genes have genealogies. Although the concepts are the same, these are actually two very different types of histories. In fact, one is contained within the other. 

The number of ancestors that a diploid individual has increases backwards in time, while the number of ancestors an individual gene copy has is each generation is always one. This has the consequence that for a given gene in an individual, most of that individual's ancestors' copies of that gene were not inherited. For example, at a specific gene in your genome, you only have two copies, meaning that of the 8 copies that existed in your 4 grandparents, 6 were lost. Your genealogy includes all four grandparents, but the gene genealogy includes only two of them (more specifically, each gene copy genealogy includes just one of them). At another gene in your genome the story is the same, but the two gene copies may trace back through two different grandparents. 

If you continue this process back further in time, to your great-great-great-great grandparents, and so on, you will find that some of your ancestors have left no gene copies at any of the genes in our genome. This the basis for the concept that ones' *genealogical ancestry does not match their genetic ancestry*. 
 
 
The figures below demonstrate this for two genes (red and blue) in a diploid individual's genome, where the two copies of each gene are shown in lighter and darker colors. 

<img src="https://raw.githubusercontent.com/eaton-lab/eaton-lab.github.io/master/slides/fundamentals/8/genealogies-1.svg" style='width:50%'>


A different gene in the genome is likely to have a different genealogical history, meaning it was inherited from a different set of ancestors.


<img src="https://raw.githubusercontent.com/eaton-lab/eaton-lab.github.io/master/slides/fundamentals/8/genealogies-2.svg" style='width:50%'>

### Gene copies 
When discussing the genealogies of genes we will use the term `gene copies` to refer to the haploid copies in a sample or population. For example, if we sampled data from 10 diploid individuals then our coalescent analysis will focus on the 20 haploid gene copies in this sample. For the purpose of this model, we do not care that these gene copies occurred in pairs within diploid individuals each generation. This is because under the assumptions of an idealized population, where there is no selection on diploid genotypes, and mating is random, the diploid phase becomes irrelevant. We still use N to refer to the size of the diploid population, and so our models use 2N to refer to the number of haploid gene copies. 


Gene copies may be identical in their DNA sequences, or they may have accumulated mutations over time such that they represent different alleles. For now, ignore the sequence of the gene copies. This is key, and often a source of confusion. Every gene copy had an ancestor in a previous generation, and if you trace their history back far enough, they will all eventually coalesce to a common ancestral copy. *Whether or not mutations occurred thoughout that history, the fact remains, there exists a genealogy for any set of gene copies*. It is easiest to first envision the genealogy, and then to imagine mutations occurring along edges of the genealogy. And this the approach we take when applying coalescent models. 

### An example genealogy
A genealogy shows the relationships of a set of samples. We will cover how to read and interpret phylogenetic trees in more detail in a later notebook, but the same concepts apply here. For now, you do not need to understand the code used to generate the genealogy. But just take note that the structure below is what we mean when referring to a genealogy. The tips represent gene copies. For example, here we sampled 6 gene copies, which we can assume came from 3 diploid individuals. 

What we will learn is that the genealogy of these six gene copies can tell us about how much genetic variation we expect to exist within and among these samples. However, we cannot observe genealogies directly. Instead, we can observe and measure genetic variation. Thus, we are often interested in the reverse inference: using the measured genetic variation to infer information about genealogies. 

In [ ]:
# an simulation of a genealogy for n samples 
mod = ipcoal.Model(tree="", Ne=1000, nsamples=6, seed=123)
mod.sim_trees(10)
mod.draw_genealogy(0);

### Coalescence
Because the number of ancestors that an individual has increases rapidly backwards in time, it becomes increasingly likely that any two individuals will share a common ancestor backwards in time. This also increases the probability that for any given gene in their genomes that they may share a gene copy that was inherited from a gene copy in that common ancestor. This is demonstrated below, where two individuals each have a light red allele that was inherited from a common ancestor they shared only a few generations back. The other allele that each possesses at this gene (dark red) was not inherited from one of their recent shared ancstors, and has a deeper coalescent time.


<img src="https://raw.githubusercontent.com/eaton-lab/eaton-lab.github.io/master/slides/fundamentals/8/genealogies-3.svg" style='width:50%'>


### The Wright-Fisher process
The Wright-Fisher model is a simple model of Mendelian Segregation of alleles and resampling of diploid genotyes over multiple generations in a **finite sized population**. 
This is important because it introduces a mathematical framework for examining the expected effect of *genetic drift* on the change in allele frequencies over time. In this way, it is nearly identical to the binomial sampling model we used in the previous notebook. However, now instead of just following the frequency of alleles over each generation, we can also keep track of the genealogies of the specific gene copies that persist over generations.


Specifically, a Wright-Fisher process is a discrete time model in which each generation is composed of 2N copies of each gene, and in each subsequent generation 2N new copies are randomly drawn from the previous generation. This is emulates a random mating process. 


As a consequence of this sampling scheme some ancestral gene copies leave zero descendants, while others may leave many more. At the extremes, a single gene copy could make up the entire next generation, or, every single gene copy could leave exactly one descendant, such that the new generation is identical to the previous one. The number of descendants a gene copy passes to the next generation tends to be between these two extremes (the average number of descendants is 1/2N). When this process is played out over multiple generations it can cause allele frequencies to change by genetic drift (as shown in the prevous notebook). It also leads to a predictable average time in generations until two gene copies  share a most recent common ancestor.


Within the history of a Wright-Fisher model population (the figure on the left below), there will always exist a genealogy that connects the history of two or more sampled gene copies in a given generation (as shown in the middle figure below). It turns out that the average time (in generations) until n samples coalesce is related to the size of the population (Ne). If the population is very large then it takes a long time on average until samples coalesce, whereas in small populations coalescence occurs very quickly. This relationship between Ne and coalescent times is the basis of the coalescent model (figure on the right below).


<img src='https://eaton-lab.org/slides/fundamentals2019/session-10-popgen/data/WF-genealogy-Drummond.png' style='width:70%'>

<p style="text-align:center"><a href='https://www.researchgate.net/figure/The-Wright-Fisher-and-Kingman-coalescent-processes-with-generation-time-r-left-K-40_fig1_2853283'>(image source)</a></p>

### The toytree and ipcoal packages
The [toytree](https://toytree.readthedocs.io) and [ipcoal](https://ipcoal.readthedocs.io) Python packages are designed to be used together within jupyter notebooks to execute interactive code to create, manipulate, and visualize tree data objects, to simulate the coalescent process, and to generate sequence data on genealogies. The `ipcoal` package is built as a wrapper around the popular `msprime` coalescent simulator, and extends its functionality for phylogenetics and for teaching. Put simply, these two tools together allow us to easily explore and learn about the coalescent.

In [ ]:
import ipcoal
import toytree

### Genealogies within a single population

Most students are familiar with phylogenetic trees, and the use of genetic data to infer phylogenetic relationships among samples. Here we are not focused on trying to infer the genealogy. Instead, our goal is **treat genealogies as a random variable**. In other words, we expect the genealogy to be different at every different gene in the genome, and we don't care exactly what the genealogy is, instead we care about the parameter that affects genealogical variation: **Ne.**

This idea is connected to the assumption of random mating within the population. If all individuals are randomly mating, then the genealogical relationships of gene copies at any gene in the genome is random. The thing that is not random, however, will be the expected **time** to until they share common ancestors. This is be determined by the effective size of the population, since it affects the probability that two samples share a common ancestor in the previous generation.

Thus, we can examine genetic variation over many sampled genes as a way of examining variation in many different genealogies. From the distribution of genetic variation we can estimate the effective population size.

Execute the code cell below several times and see how the relationships among gene copies changes in each simulation. This is the type of variation that we would expect to see at different genes in a genome. Not the y axis in units of generations. Try increasing the the variable N from 100 to 10000. How does the coalescent times change?


In [ ]:
# an simulation of a genealogy for n samples 
mod = ipcoal.Model(tree="", Ne=100, nsamples=6)
mod.sim_trees(1)
mod.draw_genealogy(0);

### Genetic variation and the coalescent

The effective population size affects the amount of genetic diversity that we expect to observe in a population ($\Theta$). This equation is based on the expected coalescent time of two samples (2N), multiplied by two because a mutation could occur on either of the descendant branches from the ancestor, and multiplied by the per-site per-generation mutation rate ($\mu$). Interestingly, in a randomly mating population we expect that heterozygosity is exactly the same as the population genetic diversity. This is because both are asking the question: what is the probability that any two randomly sampled gene copies in this population will be different (e.g., in terms of DNA, what is the chance that two randomly sampled DNA sites are different)?


 heterozygosity: $ H = 4 N_e \mu $  
 population genetic diversity (theta): $ \Theta = 4 N_e \mu $
   
   
   

### Simulating sequences on genealogies

This is demonstrated in the following simulation. Here we simulate 10K coalescent genealogies for 8 samples in a panmictic (randomly mating) population. We apply a per-site-per generation mutation rate of 1e-7 to simulate mutations on each genealogy, meaning that this is the probability in each generation that a mutation occurs on any edge, causing the descendants to inherit the mutation. This will take about a minute to run.

In [ ]:
# set variables
Ne = 10000
nsamples = 8
mut = 1e-7
nloci = 1000

# simulate sequence data
model = ipcoal.Model(tree="", Ne=Ne, nsamples=nsamples, mut=mut)
model.sim_loci(nloci=nloci, nsites=20)

We now have a sequence matrix containing data from 10K simulated genealogies. In the code below I measure the average number of differences between any two samples in the population, and also show the theoretical expectation for the population genetic diversity of this population. After completing this part once, try changing some parameter of the code above, such as increasing or decreasing the population size, and run the code again to see its effect. 

In [ ]:
# sample all combination of two gene copies
samples = list(itertools.combinations_with_replacement(range(nsamples), 2))

# measure proportion of variable sites in each sample
theta = []
idx = 0
for sample in samples:
    i, j = sample
    if i != j:
        two_samples = np.concatenate(model.seqs, 1)[[i, j], :]
        theta.append((sum(two_samples[1, :] != two_samples[0, :]) / two_samples.shape[1]))
        idx += 1

print("measured population genetic diversity (theta)")
print('mean={:.4f}, std={:.4f}'.format(np.mean(theta), np.std(theta)))

print("\ntheoretical expectation (4Neu): {}".format(4 * Ne * mut))

As you can see the amount of genetic diversity in our simulated data is very close to the expectation. This demonstration shows how the actual relationships among sampled gene copies in a panmictic population are not relevant to measuring the genetic diversity the population. We can treat the genealogy as a random variable!

This realization has many consequences that are taken advantage of in a number of additional models, including the multi-species coalescent, which is a topic of our next notebook. In this next extension we will introduce population structure, such that multiple populations exist simultaneously, and there is a potential for gene flow between populations, and to model the divergence of populations from a common ancestor. 